<a href="https://colab.research.google.com/github/hthomas229/PurpleCrown/blob/main/polars_tutorial_4_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#POLARS TUTORIAL

## **with_columns:** Adding Custom Columns to Polars DataFrames

🐻‍❄️with_columns creates a new dataframe that contains the columns from the original dataframe and the new columns according to its input expressions

In [ ]:
print(df.schema)

Add a custom column

In [ ]:
# add a column TotAtk which equals Attack + SpAtk
result = df.with_columns(
    (pl.col("Attack") + pl.col("SpAtk")).alias("TotAtk")
)
print(result)

In [ ]:
#select only Name and TotAtk -- chaining context -- we start to see pipelines
result = df.with_columns(
    (pl.col("Attack") + pl.col("SpAtk")).alias("TotAtk")
).select("Name", "TotAtk")
print(result)


Using a Variable & Adding Multiple Columns

In [ ]:
total_power = (pl.col("Attack") + pl.col("Speed")  + pl.col("Defense"))

result = df.with_columns(
    total_power = total_power,
    avg_total_power = total_power.mean(), #round for example in video
    max_total_power = total_power.max()
)

print(result)

Create a Conditioal Column with pl.when().then().otherwise()  -- Case statement

In [ ]:
#create a SpeedTier column

result = df.with_columns(
    pl.when(pl.col("Speed") >= 100).then(pl.lit("Fast"))
    .when(pl.col("Speed") >= 70).then(pl.lit("Medium Speed"))
    .otherwise(pl.lit("Slow")).alias("SpeedTier")
  )
print(result)

##Sorting and Limiting Polars DataFrames

In [ ]:
#sort pokemon by name
result =df.sort("Name") # ascending is the default argument for sort
print(result.head())

In [ ]:
#sort pokemon by Speed descending and return the top 5

result = df.sort("Speed",descending=True).limit(5)
print(result)

In [ ]:
#sort by HP descending and return the five weakest pokemon -- select only name and HP

result = df.sort("HP", descending=True).select("Name", "HP").tail(5)
print(result)

Multi Column Sort

In [ ]:
# sort by Type 1 ascending then Attack Descending -- select only Name, Type1 and Attack

result = df.sort(
    ["Type1", "Attack"],
    descending=[False, True]
).select("Name", "Type1", "Attack")

print(result)

Top and Bottom K

In [ ]:
#get 5 fastest pokemon
result = df.top_k(5, by="Speed")
print(result)

In [ ]:
#get 5 slowest pokemon
result = df.bottom_k(5, by="Speed")
print(result)

##Grouping & Aggregating Polars DataFrames

Here are some of the most commonly used aggregation functions: `sum(),mean(),median(), min(), max(), len()`



In [ ]:
# sum pokemon HP by Type1 and rename to TotHP
result = df.group_by("Type1").agg(
    pl.col("HP").sum().alias("TotHP")
)
print(result)

In [ ]:
# group by Type1 and get the Average Attack,Average Speed, Median Defense, Max HP and Min HP

result= (df.group_by("Type1")
.agg(
    pl.col("Attack").mean().alias("AvgAttack"),
    pl.col("Speed").mean().alias("AvgSpeed"),
    pl.col("Defense").median().alias("MedianDefense"),
    pl.col("HP").max().alias("MaxHP"),
    pl.col("HP").min().alias("MinHP"))
)

print(result)

In [ ]:
# get the count of each Type1 pokemon
result = df.group_by("Type1").agg(
    pl.len().alias("Count")
)

print(result)

In [ ]:
#group by speed in 10s and get max and min for each
result = df.group_by(
    (pl.col("Speed") // 10 * 10).alias("SpeedTier")
).agg(
    pl.len(),
    pl.col("Speed").max().alias("fastest"),
    pl.col("Speed").min().alias("slowest")
).sort("SpeedTier", descending=True)
print(result)